Bibliografia:

Para o modelo de submissão:
https://www.kaggle.com/code/philculliton/nlp-getting-started-tutorial/notebook

Conceitos de ML:
https://tatianaesc.medium.com/machine-learning-conceitos-e-modelos-f0373bf4f445

Visualização:
https://realpython.com/twitter-sentiment-python-docker-elasticsearch-kibana/

Conceito VADER:
https://ojs.aaai.org/index.php/ICWSM/article/view/14550/14399

1)Limpeza das palavras: tirar as stop words, lematização, Bag-of-words

## 1) Vizualização dos dados

In [1]:
import numpy as np
import pandas as pd
import nltk
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize

KeyboardInterrupt: 

In [ ]:
import gensim

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
from nltk.corpus import stopwords

In [ ]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

## 2) Minimização dos ruídos

### Text

In [ ]:
sample_tweets_train=train_df['text']
sample_tweets_test=test_df['text']

In [ ]:
contractions = {
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he shall",
"he'll've": "he shall have",
"he's": "he has",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has",
"i'd": "I had",
"i'd've": "I would have",
"i'll": "I shall",
"i'll've": "I shall have",
"i'm": "I am",
"i've": "I have",
"it's": "it is",
"isn't": "is not",
"it'd": "it had",
"it'd've": "it would have",
"it'll": "it shall",
"it'll've": "it shall have",
"it's": "it has",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had",
"she'd've": "she would have",
"she'll": "she shall",
"she'll've": "she shall have",
"she's": "she has",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that has",
"there'd": "there had",
"there'd've": "there would have",
"there's": "there has",
"they'd": "they had",
"they'd've": "they would have",
"they'll": "they shall",
"they'll've": "they shall have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall",
"what'll've": "what shall have",
"what're": "what are",
"what's": "what has",
"what've": "what have",
"when's": "when has",
"when've": "when have",
"where'd": "where did",
"where's": "where has",
"where've": "where have",
"who'll": "who shall",
"who'll've": "who shall have",
"who's": "who has",
"who've": "who have",
"why's": "why has",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had",
"you'd've": "you would have",
"you'll": "you shall",
"you'll've": "you shall have",
"you're": "you are",
"you've": "you have"
}

In [ ]:
def contracao(sample_tweets):
    sample = []
    for tweets in sample_tweets:
        for word in tweets.split():
            if word.lower() in contractions:
                tweets = tweets.replace(word, contractions[word.lower()])
        sample.append(tweets)
    return sample

In [ ]:
sample_tweets_train = pd.DataFrame(contracao(sample_tweets_train))
sample_tweets_test = pd.DataFrame(contracao(sample_tweets_test))

In [ ]:
sample_tweets_train.head()

,0
0,our deeds are the reason of this earthquake ma...
1,forest fire near la ronge sask canada
2,all residents asked to shelter in place are be...
3,people receive wildfires evacuation orders in...
4,just got sent this photo from ruby alaska as s...


In [ ]:
def limpeza(tweets):
    tweets = tweets.str.replace(r'@[a-zA-Z0-9_!.]{0,25}','')
    tweets = tweets.str.replace('(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/(?:[-\w.]|(?:%[\da-fA-F]{2}))+','')
    tweets = tweets.str.replace(r'[0-9]','')
    tweets = tweets.replace("\\x89Û_",'')
    tweets = tweets.replace('\x89ÛÓ','')
    tweets = tweets.replace('\x89ÛÓ','')
    tweets = tweets.replace('\x89ÛÒ','')
    tweets = tweets.replace('\x89Û','')
    tweets = tweets.replace('\x89Û÷','')
    tweets = tweets.replace('\x89ûï','')
    tweets = tweets.replace('u','you')
    #retirada de símbolos não alfanuméticos 
    tweets = tweets.str.replace(r"['#=<>,.;:_?!(-)\[\]]","")
    tweets = tweets.str.lower()
    return tweets

In [ ]:
sample_tweets_train = limpeza(sample_tweets_train[0])
sample_tweets_test = limpeza(sample_tweets_test[0])

C:\Users\yfrom\AppData\Local\Temp\ipykernel_11196\2165951979.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets = tweets.str.replace(r'@[a-zA-Z0-9_!.]{0,25}','')
C:\Users\yfrom\AppData\Local\Temp\ipykernel_11196\2165951979.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets = tweets.str.replace('(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/(?:[-\w.]|(?:%[\da-fA-F]{2}))+','')
C:\Users\yfrom\AppData\Local\Temp\ipykernel_11196\2165951979.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets = tweets.str.replace(r'[0-9]','')
C:\Users\yfrom\AppData\Local\Temp\ipykernel_11196\2165951979.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets = tweets.str.replace(r"['#=<>,.;:_?!(-)\[\]]","")


In [ ]:
stop_words = set(stopwords.words("english"))

In [ ]:
def lematizar(tweets):
    sample_tweets= []
    for tweet in tweets:
        filtered_list = []
        words_lemmatized = []
        words = []
        words = word_tokenize(tweet) 
        words_lemmatized = [lemmatizer.lemmatize(word) for word in words] 
        for word in words_lemmatized:
            if word not in stop_words:
                filtered_list.append(word)
        sample_tweets.append(filtered_list)
    return sample_tweets    

In [ ]:
sample_tweets_train = lematizar(sample_tweets_train)
sample_tweets_test = lematizar(sample_tweets_test)

In [ ]:
train_df['words']= sample_tweets_train
test_df['words']= sample_tweets_test

- tratamento:
    - de palavras erradas -> biblioteca usada no repositório abi-academy-hack
    - letras repetidas 

In [ ]:
test_df = test_df.drop(columns=['location', 'keyword','id','text'])
train_df = train_df.drop(columns=['location', 'keyword','id','text'])

In [ ]:
train_df['text'] = train_df['words'].apply(lambda x: ' '.join([str(elem) for elem in x]))
test_df['text'] = test_df['words'].apply(lambda x: ' '.join([str(elem) for elem in x]))

In [ ]:
train_df['text'][3:20]

3     people receive wildfire evacuation order calif...
4     got sent photo ruby alaska smoke wildfire pour...
5     rockyfire update california hwy closed directi...
6     flood disaster heavy rain cause flash flooding...
7                                top hill see fire wood
8     ha emergency evacuation happening building acr...
9                            afraid tornado coming area
10                      three people died heat wave far
11    haha south tampa getting flooded hah- wait sec...
12    raining flooding florida tampabay tampa day lo...
13                      flood bago myanmar arrived bago
14           damage school bus multi car crash breaking
15                                               ha man
16                                           love fruit
17                                        summer lovely
18                                             car fast
19                                      goooooooaaaaaal
Name: text, dtype: object

## Bag-of-words
 Definição: Bag-of-words model is a way of representing text data when modeling text with machine learning algorithms. Machine learning algorithms cannot work with raw text directly; the text must be converted into well defined fixed-length(vector) numbers.

Formação da Bag-of-word para treino e teste com count vectorizer

In [ ]:
treino = train_df.head(6090)
teste = train_df.tail(1523)

In [ ]:
target_treino= treino['target']
traget_teste= teste['target']

In [ ]:
Vectorizer = CountVectorizer()
count_treino_cv = Vectorizer.fit_transform(treino['text'])
count_teste_cv = Vectorizer.transform(teste['text'])

In [ ]:
count_treino_cv[:10,:10]

<10x10 sparse matrix of type '<class 'numpy.int64'>'
	with 0 stored elements in Compressed Sparse Row format>

Formação da Bag-of-word para treino e teste com tf-idf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
count_treino_tfidf = tfidf_vectorizer.fit_transform(treino['text'])
count_teste_tfidf = tfidf_vectorizer.transform(teste['text'])

word2vec

In [ ]:
word2vec_path = "GoogleNews-vectors-negative300.bin.gz"
word2vec = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

## Modelos de classificação

In [ ]:
def metricas(y_test,y_predicted):
    precision = precision_score(y_test, y_predicted, pos_label=None,average='weighted')             
    recall = recall_score(y_test, y_predicted, pos_label=None,average='weighted')
    f1 = f1_score(y_test, y_predicted, pos_label=None, average='weighted')
    accuracy = accuracy_score(y_test, y_predicted)
    return accuracy, precision, recall, f1


### Regressão Logística

GridSearch

In [ ]:
clf_cv = LogisticRegression(C=1, class_weight='balanced', solver='sag',multi_class='auto', n_jobs=-1, random_state=40,max_iter= 100000)
clf_cv.fit(count_treino_cv, target_treino)
Predição_target_cv = clf_cv.predict(count_teste_cv)

In [ ]:
print(metricas(traget_teste, Predição_target_cv))

(0.7879185817465528, 0.7896749023950892, 0.7879185817465528, 0.7865201956024487)


In [ ]:
clf_tfidf = LogisticRegression(C=1, class_weight='dict', solver='newton-cg',multi_class='auto', n_jobs=-1, random_state=40,max_iter= 1000)
clf_tfidf.fit(count_treino_tfidf, target_treino)
Predição_target_tfidf = clf_tfidf.predict(count_teste_tfidf)

In [ ]:
print(metricas(traget_teste, Predição_target_tfidf))
#Sem parâmetros ideias:
#(0.7912015758371634, 0.7911097527493921, 0.7912015758371634, 0.7908037193499462)
#(0.7925147734734077, 0.8075907785931117, 0.7925147734734077, 0.7875804286920929)

(0.788575180564675, 0.8037166405742918, 0.788575180564675, 0.7834546624011325)


SVM

In [ ]:
parameters = [{
'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 
'C': [1,2,3,300,500],
'max_iter': [1000,100000],
'gamma':['scale', 'auto'],
'class_weight':['dict' ,'balanced']
}]

In [ ]:
clf = GridSearchCV(
        SVC(), parameters, scoring='accuracy'
    )
clf.fit(count_treino_tfidf, target_treino)

In [ ]:
print(clf.best_params_)

{'C': 1, 'class_weight': 'balanced', 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': 100000}


In [ ]:
clf = SVC(kernel='rbf',C=1,class_weight='balanced',gamma='scale', max_iter=100000) 
clf.fit(count_treino_cv, target_treino)
Predição_target_SVM_cv = clf.predict(count_teste_cv)

In [ ]:
print(metricas(traget_teste, Predição_target_SVM_cv))
#antes do grid search: (0.7544320420223244, 0.7607494234790165, 0.7544320420223244, 0.7505317206525575)

(0.7820091923834537, 0.7990065430270551, 0.7820091923834537, 0.7761385531145388)


In [ ]:
clf = SVC(kernel='rbf',C=1,class_weight='balanced',gamma='scale', max_iter=100000) 
clf.fit(count_treino_tfidf, target_treino)
Predição_target_SVM_tfidf= clf.predict(count_teste_tfidf)

In [ ]:
print(metricas(traget_teste, Predição_target_SVM_tfidf))

(0.7820091923834537, 0.7866863652510794, 0.7820091923834537, 0.7794820049168832)
